In [1]:
# pip install gensim
import gensim
import nltk
import pandas as pd 

from nltk.stem import SnowballStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import sent_tokenize, word_tokenize
from gensim import corpora, models 

texto = open("../../Tascas/Data/ejemplo.txt", "r").read()
frases = sent_tokenize(texto)
frases = pd.DataFrame({"Frases":frases})


In [2]:
stemmer = SnowballStemmer("english")

def preproceso(texto):
    resultado=[]
    for token in gensim.utils.simple_preprocess(texto):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            resultado.append(stemmer.stem(token))
    return resultado

frasesprocesadas = frases["Frases"].map(preproceso)

In [3]:
frasesprocesadas

0                        [apach, http, server, project]
1     [apach, http, server, project, collabor, softw...
2     [project, joint, manag, group, volunt, locat, ...
3                    [project, apach, softwar, foundat]
4     [addit, hundr, user, contribut, idea, code, do...
5     [file, intend, briefli, histori, apach, http, ...
6     [apach, came, februari, popular, server, softw...
7     [develop, httpd, stall, left, ncsa, webmast, d...
8     [small, group, webmast, contact, privat, mail,...
9     [brian, behlendorf, cliff, skolnick, mail, lis...
10    [februari, core, contributor, form, foundat, o...
11    [coincid, ncsa, restart, develop, period, bran...
12    [earli, apach, server, knew, codebas, need, ge...
13    [june, hartil, rest, group, focus, implement, ...
14    [group, switch, server, base, juli, ad, featur...
15    [extens, beta, test, port, obscur, platform, d...
16    [year, group, form, apach, server, pass, ncsa,...
17    [member, apach, group, form, apach, softwa

In [4]:
dictionary = gensim.corpora.Dictionary(frasesprocesadas)
dictionary.filter_extremes(no_below=3, no_above=0.5, keep_n=100000)

corpus = [dictionary.doc2bow(doc) for doc in frasesprocesadas]


In [5]:
corpus  # tods ls plabras que aparecen

[[(0, 1), (1, 1), (2, 1), (3, 1)],
 [(0, 1),
  (1, 2),
  (2, 1),
  (3, 2),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1)],
 [(2, 1), (3, 1), (7, 1), (13, 1), (14, 1), (15, 1), (16, 1)],
 [(0, 1), (2, 1), (11, 1), (17, 1)],
 [(2, 1), (5, 1), (14, 1), (18, 1), (19, 1), (20, 1)],
 [(0, 1), (1, 1), (3, 1), (21, 1)],
 [(0, 1), (1, 1), (3, 1), (7, 1), (11, 1), (22, 1)],
 [(7, 2), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1)],
 [(15, 1), (28, 1), (29, 1), (30, 1), (31, 1)],
 [(7, 1), (30, 1), (32, 1), (33, 1), (34, 1)],
 [(0, 2),
  (3, 2),
  (15, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (21, 1),
  (22, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (29, 1),
  (32, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 1)],
 [(2, 1), (3, 1), (7, 2), (24, 1), (26, 2), (34, 1), (39, 1)],
 [(0, 1), (3, 1), (27, 1)],
 [(0, 1),
  (3, 1),
  (5, 1),
  (8, 1),
  (9, 1),
  (15, 1),
  (20, 1),
  (23, 1),
  (36, 1),
  (40, 1),
  (41, 1)],
 [(0, 1), (3, 1), (8, 1), (15, 1), (35, 1)

In [6]:
tfidf = models.TfidfModel(corpus)  # modelo alternativo
corpus_tfidf = tfidf[corpus]       # frecuencias terminos inverso frecuencia documentos

In [9]:
# Primer modelo  # encontrar los temas en el documento
lda_model = gensim.models.LdaMulticore(corpus, num_topics=10, id2word=dictionary, passes=5, workers=3)
for idx, topic in lda_model.print_topics(-1):
    print('Tema: {} \nPalabras: {}\n'.format(idx, topic))

Tema: 0 
Palabras: 0.321*"compani" + 0.253*"protocol" + 0.070*"small" + 0.006*"individu" + 0.006*"avail" + 0.006*"robust" + 0.006*"implement" + 0.006*"free" + 0.006*"apach" + 0.006*"project"

Tema: 1 
Palabras: 0.109*"develop" + 0.074*"project" + 0.067*"repositori" + 0.066*"sourc" + 0.064*"code" + 0.055*"apach" + 0.051*"inform" + 0.049*"access" + 0.042*"group" + 0.035*"committ"

Tema: 2 
Palabras: 0.182*"list" + 0.135*"mail" + 0.132*"activ" + 0.092*"member" + 0.077*"group" + 0.075*"work" + 0.041*"usual" + 0.041*"contribut" + 0.041*"communic" + 0.004*"peopl"

Tema: 3 
Palabras: 0.165*"apach" + 0.135*"server" + 0.134*"http" + 0.060*"softwar" + 0.054*"develop" + 0.037*"featur" + 0.037*"commerci" + 0.037*"project" + 0.024*"form" + 0.019*"extens"

Tema: 4 
Palabras: 0.150*"softwar" + 0.122*"compani" + 0.098*"apach" + 0.064*"individu" + 0.060*"support" + 0.057*"provid" + 0.034*"list" + 0.034*"small" + 0.034*"public" + 0.034*"contribut"

Tema: 5 
Palabras: 0.118*"softwar" + 0.103*"free" + 0.0

In [10]:
# Segundo modelo
lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=5, workers=3)
for idx, topic in lda_model.print_topics(-1):
    print('Tema: {} \nPalabras: {}\n'.format(idx, topic))

Tema: 0 
Palabras: 0.061*"ad" + 0.057*"form" + 0.052*"server" + 0.048*"result" + 0.047*"featur" + 0.044*"addit" + 0.043*"ncsa" + 0.042*"group" + 0.041*"base" + 0.040*"httpd"

Tema: 1 
Palabras: 0.109*"list" + 0.078*"mail" + 0.064*"communiti" + 0.057*"releas" + 0.049*"communic" + 0.035*"avail" + 0.032*"peopl" + 0.032*"particular" + 0.032*"usual" + 0.030*"fix"

Tema: 2 
Palabras: 0.082*"free" + 0.055*"compani" + 0.054*"apach" + 0.053*"group" + 0.050*"small" + 0.043*"contributor" + 0.042*"core" + 0.038*"individu" + 0.032*"server" + 0.031*"communic"

Tema: 3 
Palabras: 0.090*"project" + 0.090*"inform" + 0.088*"http" + 0.074*"work" + 0.071*"httpd" + 0.061*"apach" + 0.046*"need" + 0.042*"server" + 0.026*"develop" + 0.024*"base"

Tema: 4 
Palabras: 0.210*"protocol" + 0.168*"compani" + 0.061*"small" + 0.010*"apach" + 0.010*"softwar" + 0.010*"server" + 0.010*"http" + 0.010*"project" + 0.010*"group" + 0.010*"foundat"

Tema: 5 
Palabras: 0.084*"provid" + 0.074*"support" + 0.070*"contribut" + 0.03